In [84]:
import torch
import time
import gc #force garbage collection if necessary

In [15]:
#find GPU device
if torch.cuda.is_available():
    #if Nvidia GPU available, use it
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    #if apple silicon available
    device = torch.device('mps')
else:
    #else just go with CPU
    device = torch.device('cpu')

print(f"Program using {device}")

Program using mps


In [96]:
class Gate:

    #
    H = torch.tensor([[1., 1.],
                      [1., -1.]], device=device, dtype=torch.cfloat) / 2**0.5
    
    I = torch.tensor([[1., 0.],
                      [0., 1.]], device=device, dtype=torch.cfloat)

    S = torch.tensor([[1., 0.],
                      [0., 1.j]], device=device, dtype=torch.cfloat)

    SX= torch.tensor([[1.+1.j, 1.-1.j],
                      [1.-1.j, 1.+1.j]], device=device, dtype=torch.cfloat) / 2
    
    X = torch.tensor([[0., 1.],
                      [1., 0.]], device=device, dtype=torch.cfloat)

    Y = torch.tensor([[0., -1.j],
                      [1.j, 0.]], device=device, dtype=torch.cfloat)

    Z = torch.tensor([[1., 0.],
                      [0., -1.]], device=device, dtype=torch.cfloat)
    
    def __init__(self, custom=None):
        self.custom = custom

    #---Single qubit matrix operator---#
    '''
    Routine to apply given single-qubit unitary matrix operator on a target qubit index
    in some given states vector (representing the wavefunction)
    '''
    def apply(gate, states, target):
        if target(
    
    #---Phase transforms---#
    
    '''
    Ph (Global phase transform) takes a real-valued angle argument
    representing a complex phase rotation in the Block sphere
    The phase transform rotates the qubit state without changing its
    probability of collapse to |0) or |1)
    '''
    def Ph(self, phase):
        phase = phase % (2*torch.pi) #modulus 2*pi of input
        Ph_angle = torch.tensor([0. + 1j*(phase)], device=device, dtype=torch.cfloat)
        Ph_angle = torch.exp(Ph_angle)
        return Ph_angle * self.I
    '''
    Bloch sphere rotation around the x axis
    '''
    def Rx(self, angle):
        angle = torch.tensor(angle, device=device)
        #form sin and cos components
        cos_comp = torch.cos((angle/2))*self.I
        sin_comp = 1j*torch.sin((angle/2))*self.X
        #form complete matrix
        Rx_rot = cos_comp - sin_comp
        return Rx_rot
    '''
    Bloch sphere rotation around the y axis
    '''
    def Ry(self, angle):
        angle = torch.tensor(angle, device=device)
        #form sin and cos components
        cos_comp = torch.cos((angle/2))*self.I
        sin_comp = 1j*torch.sin((angle/2))*self.Y
        #form complete matrix
        Ry_rot = cos_comp - sin_comp
        return Ry_rot
    '''
    Bloch sphere rotation around the z axis
    '''
    def Rz(self, angle):
        angle = torch.tensor(angle, device=device)
        #form sin and cos components
        cos_comp = torch.cos(angle/2)*self.I
        sin_comp = 1j*torch.sin(angle/2)*self.Z
        #form complete matrix
        Rz_rot = cos_comp - sin_comp
        return Rz_rot
    '''
    Universal Bloch sphere rotation
    U(theta, phi, lambda)
    '''
    def U(self, theta, phi, lam):
        #convert angles to tensors
        theta_t = torch.tensor(theta/2, device=device)
        phi_t = torch.tensor(phi, device=device)
        lam_t = torch.tensor(lam, device=device)
        #form tensor using closed-form definition
        U_rot = torch.tensor(
            [[  torch.cos(theta_t),
                -1*torch.exp(1.j*lam_t)*torch.sin(theta_t)
             ],
             [  torch.exp(1.j*phi_t)*torch.sin(theta_t),
                torch.exp(1.j*(lam_t+phi_t))*torch.cos(theta_t)
             ]], device=device)
        return U_rot

    #---Controlled gates---#
    '''
    These sets of operators are syntactically different than
    2x2 and rotation operators as they must extend between
    a control qubit and a target qubit
    For a n-qubit system with target 4 and control 0, the
    operator must extend across 5 qubits! Furthermore, the
    matrix created for an n-qubit system holds 2^n x 2^n values
    Since for all practical purposes this is vastly inefficient,
    these routines use classical bitwise analogs instead of matrices
    '''
    def CNOT(states, control, target):
        C = 1 << control
        T = 1 << target
        N = len(states)
        if (control < 0) or (target < 0):
            raise ValueError("Indices cannot be negative")
        elif (2**control > N) or (2**target > N):
            raise ValueError("Control and target qubits must be in range")
        #perform XOR on target and state indices
        for i in range(N):
            if i & C:
                j = i^T
                if j>i:
                    states[i], states[j] = states[j], states[i]
        return states

    #---Non-clifford gates---#
    '''
    These are a set of 

gate = Gate()
gate.Rz(torch.pi/4)

tensor([[0.9239-0.3827j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.9239+0.3827j]], device='mps:0')

In [143]:
print([bin(i&1) for i in range(8)])

['0b0', '0b1', '0b0', '0b1', '0b0', '0b1', '0b0', '0b1']


In [148]:
def apply_cnot(states, control, target):
    C = 1 << control
    T = 1 << target
    N = len(states)
    for i in range(N):
        if i & C:
            j = i^T
            if j>i:
                states[i], states[j] = states[j], states[i]
    return states

s = [0.707, 0.707, 0.707, 0.707, 0, 0, 0, 0]
apply_cnot(s, 0, 2)

[0.707, 0, 0.707, 0, 0, 0.707, 0, 0.707]

In [166]:
def gpu_mem_usage():
    if torch.cuda.is_available():
        mem_used = torch.cuda.memory_allocated()
    elif torch.backends.mps.is_available():
        mem_used = torch.mps.current_allocated_memory()
    else:
        print("Cannot show memory usage for CPU")
        return
    mem_used /= 1024**2
    return mem_used